### r02 再計算 実験可視化  

In [ ]:
# coding:utf-8
import json
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import glob
import seaborn as sns
import pickle
import math
from collections import OrderedDict

In [ ]:
# 解析基準位置に移動する関数
def move_root_dir(root_dir):
    while not (os.path.basename(os.getcwd())== root_dir):
        os.chdir('../')
    print('moving to:',os.path.basename(os.getcwd()))
    return os.getcwd()

In [ ]:
work_code='20201012_preprocess_data'
root_dir = '2020_moox_morikoro_project'
root_path = move_root_dir(root_dir)
input_data_dir = 'input/20201012_raw_data'
save_csv_dir = 'preprocess_data/'+work_code+'/csvfile'
if not os.path.isdir(save_csv_dir):
    os.makedirs(save_csv_dir)
save_pkl_dir= 'preprocess_data/'+work_code+'/pkl'
if not os.path.isdir(save_pkl_dir):
    os.makedirs(save_pkl_dir)
save_img_dir= 'output/20201012_output_data/fig/'+work_code
if not os.path.isdir(save_img_dir):
    os.makedirs(save_img_dir)

In [ ]:
os.chdir('src/preprocess/20201012_script')

from moox_detect_action.Detect_rule.DetectBodyStatus.detect_look_points import DetectLookPoints
look_points = DetectLookPoints(ids='2')
look_points2 = DetectLookPoints(ids='2')
look_points3 = DetectLookPoints(ids='3')
from moox_detect_action.Detect_rule.DetectBodyStatus.detect_face_direction_for_azure import DetectFaceDirection
face_direction = DetectFaceDirection()

root_path = move_root_dir(root_dir)

In [ ]:
# 関数ボックス
def read_csv_file(csv_file):
    df = pd.read_csv(csv_file)
    return df

def fill_data(df_data, st, fn):
    df = df_data.copy()
    df['Timestamp'] =pd.to_datetime(df['sensing_time'])
    df = df.set_index('Timestamp')
    df = df.resample('100ms').ffill()
    df = df.reset_index(drop=False)     
    df = df[(df['Timestamp'] > pd.to_datetime(st)) & (df['Timestamp'] <= pd.to_datetime(fn)) ]
    if(len(df)<1):
        return False, df_data        
    df = df.reset_index(drop=False)
    return True, df  

def get_data_75_3_body(df,i,head='data_body_'):
    axt = ['x', 'y', 'z']
    convert_list = [
        ['pelvis', 0],
        ['naval', 1],
        ['chest', 2],
        ['neck', 3],
        ['l_clavicle', 4],
        ['l_shoulder', 5],
        ['l_elbow', 6],
        ['l_wrist', 7],
        ['l_hand', 8],
        ['l_handtip', 9],
        ['l_thumb', 10],
        ['r_clavicle', 11],
        ['r_shoulder', 12],
        ['r_elbow', 13],
        ['r_wrist', 14],
        ['r_hand', 15],
        ['r_handtip', 16],
        ['r_thumb', 17],
        ['l_hip', 18],
        ['l_knee', 19],
        ['l_ankle', 20],
        ['l_foot', 21],
        ['r_hip', 22],
        ['r_knee', 23],
        ['r_ankle', 24],
        ['r_foot', 25],
        ['head', 26],
        ['nose', 27],
        ['l_eyes', 28],
        ['l_ear', 29],
        ['r_eyes', 30],
        ['r_ear', 31],
    ]
    body_dict={}
    body_dict['sensing_time']=df.at[i,'sensing_time']
    
    for cols in convert_list:
        body_dict[cols[0]]={}
        for ax in axt:
            col = head+cols[0]+'_'+ax
            body_dict[cols[0]][ax]=df.at[i,col]  
    return body_dict

def set_direction_data(df,i,data_dict,header):
    cols_list=[
        'face_direction_horizontal',
        'face_direction_vertical',
        'face_direction_bace_x','face_direction_bace_y','face_direction_bace_z',
        'face_direction_bace_right_eyes_x',
        'face_direction_bace_right_eyes_y',
        'face_direction_bace_right_eyes_z',
        'face_direction_bace_left_eyes_x',
        'face_direction_bace_left_eyes_y',
        'face_direction_bace_left_eyes_z'
    ]    
    for col in cols_list:
        colt =header+col
        df.at[i,colt]=data_dict[col]
    return df

def set_look_data(df,i,data_dict,header):
    cols_list=[
        'look_point_left_x','look_point_left_y','look_point_left_z',
        'look_point_left_object','look_point_left_distance',
        'look_point_left_eyes_x','look_point_left_eyes_y','look_point_left_eyes_z',  
        'look_point_left_eyes_object','look_point_left_eyes_distance',
        'look_point_center_x','look_point_center_y','look_point_center_z',          
        'look_point_center_object','look_point_center_distance',
        'look_point_right_eyes_x','look_point_right_eyes_y','look_point_right_eyes_z',
        'look_point_right_eyes_object','look_point_right_eyes_distance',
        'look_point_right_x','look_point_right_y','look_point_right_z',
        'look_point_right_object','look_point_right_distance',
        'look_point_top_x','look_point_top_y','look_point_top_z',
        'look_point_top_object','look_point_top_distance',
        'look_point_bottom_x','look_point_bottom_y','look_point_bottom_z',
        'look_point_bottom_object','look_point_bottom_distance',
    ]    
    for col in cols_list:
        colt =header+col
        df.at[i,colt]=data_dict[col]
    return df  

def calculate_data(df_data, offset_h=0.0, offset_v=0.0, head='data_body_'):
    # 補正
    look_points.set_offset(offset_h=offset_h, offset_v=offset_v)
    # 処理
    df = df_data.copy()    
    for i in range(len(df_data)):
        # 10 Hz 器官点データ取得
        body_dict = get_data_75_3_body(df, i, head=head)
        
        # 再計算(顔向き)
        face_direction.Calculate(body_dict, is_data=True)
        body_status_dict_r = face_direction.output_data
        df = set_direction_data(df,i,body_status_dict_r,'r_')
        
        # 再計算(視点)
        look_points.Calculate(body_status_dict_r, is_data=True)
        look_points_dict_r= look_points.output_data
        df = set_look_data(df,i,look_points_dict_r,'r_')
    return df

def side_block_9(xmax,xmin,ymax,ymin,zmax,zmin):
    box_9=[]

    xmax_=max([xmax,xmin])
    xmin_=min([xmax,xmin])
    xmax = xmax_
    xmin = xmin_
    ymax_=max([ymax,ymin])
    ymin_=min([ymax,ymin])
    ymax = ymax_
    ymin = ymin_
    zmax_=max([zmax,zmin])
    zmin_=min([zmax,zmin])
    zmax = zmax_
    zmin = zmin_    
    
    x03= xmin
    y03= ymin
    z03= zmin
    x13= xmin + int(abs(xmax-xmin)/3)
    y13= ymin + int(abs(ymax-ymin)/3)
    z13= zmin + int(abs(zmax-zmin)/3)
    x23= xmin + int(abs(xmax-xmin)*2/3)
    y23= ymin + int(abs(ymax-ymin)*2/3)
    z23= zmin + int(abs(zmax-zmin)*2/3)
    x33= xmax
    y33= ymax
    z33= zmax

    # 1
    cx= int((x23 + x33)/2.)
    cy= int((y23 + y33)/2.)
    cz= int((z23 + z33)/2.)
    box_9.append([cy,cy,cz, x23,y23,z23, x23,y33,z33, x33,y23,z23, x33,y33,z33, ])    
    # 2
    cx= int((x13 + x23)/2.)
    cy= int((y23 + y33)/2.)
    cz= int((z23 + z33)/2.)
    box_9.append([cy,cy,cz, x13,y23,z23, x13,y33,z33, x23,y23,z23, x23,y33,z33, ])
    # 3
    cx= int((x03 + x13)/2.)
    cy= int((y23 + y33)/2.)
    cz= int((z23 + z33)/2.)
    box_9.append([cy,cy,cz, x03,y23,z23, x03,y33,z33, x13,y23,z23, x13,y33,z33, ])    

    # 4
    cx= int((x23 + x33)/2.)
    cy= int((y13 + y23)/2.)
    cz= int((z13 + z23)/2.)
    box_9.append([cy,cy,cz, x23,y13,z13, x23,y23,z23, x33,y13,z13, x33,y23,z23, ])
    # 5
    cx= int((x13 + x23)/2.)
    cy= int((y13 + y23)/2.)
    cz= int((z13 + z23)/2.)
    box_9.append([cy,cy,cz, x13,y13,z13, x13,y23,z23, x23,y13,z13, x23,y23,z23, ])
    # 6
    cx= int((x03 + x13)/2.)
    cy= int((y13 + y23)/2.)
    cz= int((z13 + z23)/2.)    
    box_9.append([cy,cy,cz, x03,y13,z13, x03,y23,z23, x13,y13,z13, x13,y23,z23, ])    

    # 7
    cx= int((x23 + x33)/2.)
    cy= int((y03 + y13)/2.)
    cz= int((z03 + z13)/2.)
    box_9.append([cy,cy,cz, x23,y03,z03, x23,y13,z13, x33,y03,z03, x33,y13,z13, ])    
    # 8
    cx= int((x13 + x23)/2.)
    cy= int((y03 + y13)/2.)
    cz= int((z03 + z13)/2.)
    box_9.append([cy,cy,cz, x13,y03,z03, x13,y13,z13, x23,y03,z03, x23,y13,z13, ])
    # 9
    cx= int((x03 + x13)/2.)
    cy= int((y03 + y13)/2.)
    cz= int((z03 + z13)/2.)
    box_9.append([cy,cy,cz, x03,y03,z03, x03,y13,z13, x13,y03,z03, x13,y13,z13, ])

    return box_9

# 描画関数
def plot_object(plt,axis_x=0,axis_y=1,color='Glay'):
    object_tank = look_points.detect_cross.object_tank

    for object_box in object_tank:
        xbox=[]
        ybox=[]
        if(object_box[4][0]=='roof_body' or object_box[4][0]=='floor_body' or
          object_box[4][0]=='left_body' or #object_box[4][0]=='back_screen' or
          object_box[4][0]=='front_body' or object_box[4][0]=='rear_body' ):
            continue
        for i in range(4):
            box = object_box[i]
            xbox.append(box[axis_x])
            ybox.append(box[axis_y])        
            plt.scatter(box[axis_x],box[axis_y],marker='.',c='Gray',alpha=0.5)
        xmax = np.max(xbox)
        xmin = np.min(xbox)
        ymax = np.max(ybox)
        ymin = np.min(ybox)
        plt = frame_plot(plt, xmin=xmin, xmax=xmax,ymin=ymin,ymax=ymax,color="Gray",alpha=0.9)
        if(object_box[4][0]=='side_screen' or object_box[4][0]=='back_screen'):
            plt = cross_frame_plot_9(plt, xmin=xmin, xmax=xmax,ymin=ymin,ymax=ymax,fcolor="Gray",ccolor='Gray',alpha=0.5)
            
            
    return plt

def cross_frame_plot_9(plt,xmin=0, xmax=0,ymin=0,ymax=0,fcolor="Gray",ccolor='R',alpha=0.9):
    xmax_=max([xmax,xmin])
    xmin_=min([xmax,xmin])
    xmax = xmax_
    xmin = xmin_
    ymax_=max([ymax,ymin])
    ymin_=min([ymax,ymin])
    ymax = ymax_
    ymin = ymin_
    x03= xmin
    y03= ymin
    x13= xmin + int(abs(xmax-xmin)/3)
    y13= ymin + int(abs(ymax-ymin)/3)
    x23= xmin + int(abs(xmax-xmin)*2/3)
    y23= ymin + int(abs(ymax-ymin)*2/3)
    x33= xmax
    y33= ymax    
    # left line
    plt = cross_plot(plt, xmin=x03, xmax=x13,ymin=y03,ymax=y13,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=x03, xmax=x13,ymin=y13,ymax=y23,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=x03, xmax=x13,ymin=y23,ymax=y33,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    #print(x03,x13,x23,x33)
    #print(y03,y13,y23,y33)    
    # center line
    plt = cross_plot(plt, xmin=x13, xmax=x23,ymin=y03,ymax=y13,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=x13, xmax=x23,ymin=y13,ymax=y23,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=x13, xmax=x23,ymin=y23,ymax=y33,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    # right line
    plt = cross_plot(plt, xmin=x23, xmax=x33,ymin=y03,ymax=y13,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=x23, xmax=x33,ymin=y13,ymax=y23,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=x23, xmax=x33,ymin=y23,ymax=y33,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    return plt

def cross_frame_plot_4(plt,xmin=0, xmax=0,ymin=0,ymax=0,fcolor="Gray",ccolor='R',alpha=0.9):
    xc=(xmax+xmin)/2
    yc=(ymax+ymin)/2
    plt = cross_plot(plt, xmin=xmin, xmax=xc,ymin=ymin,ymax=yc,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=xmin, xmax=xc,ymin=yc,ymax=ymax,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=xc, xmax=xmax,ymin=ymin,ymax=yc,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    plt = cross_plot(plt, xmin=xc, xmax=xmax,ymin=yc,ymax=ymax,fcolor=fcolor,ccolor=ccolor,alpha=alpha)
    return plt

def cross_plot(plt,xmin=0, xmax=0,ymin=0,ymax=0,fcolor="Gray",ccolor='R',alpha=0.9):
    # cross
    plt.plot([xmax, xmin],[ymax, ymin], ccolor, alpha=alpha)
    plt.plot([xmin, xmax],[ymax, ymin], ccolor, alpha=alpha)
    # frame
    plt = frame_plot(plt, xmin=xmin, xmax=xmax,ymin=ymin,ymax=ymax,color=fcolor,alpha=alpha)
    return plt
    
def frame_plot(plt,xmin=0, xmax=0,ymin=0,ymax=0,color="Gray",alpha=0.9):
    # frame
    plt.plot([xmin, xmin],[ymin, ymax], c=color, alpha=alpha)
    plt.plot([xmin, xmax],[ymin, ymin], c=color, alpha=alpha)
    plt.plot([xmax, xmax],[ymin, ymax], c=color, alpha=alpha)
    plt.plot([xmin, xmax],[ymax, ymax], c=color, alpha=alpha)    
    return plt

def plot_fig_body(body_dict,plt,axis_x=0,axis_y=1,color='R'):
    cols=['l_eyes','l_ear','r_eyes','r_ear','head','neck','nose','chest','naval','pelvis',]
    axt = ['x', 'y', 'z']
    joint_ids=[[0,1],[2,3],[0,2],[0,6],[2,6],[6,4],
               [4,5],[5,7],[7,8],[8,9],]
    joint_pos = []
    for col in cols:
        box=[]
        for ax in axt:
            box.append(body_dict[col][ax])
        joint_pos.append(box)
    
    # body plot
    for pos in joint_pos:
        plt.scatter(pos[axis_x],pos[axis_y],marker='o',c=color,alpha=0.2)    
    for box in joint_ids:       
        #plt.plot([joint_pos[box[0]][axis_x], joint_pos[box[1]][axis_x]],
        #         [joint_pos[box[0]][axis_y], joint_pos[box[1]][axis_y]], color, alpha=0.1)
        plt.plot([joint_pos[box[0]][axis_x], joint_pos[box[1]][axis_x]],
                 [joint_pos[box[0]][axis_y], joint_pos[box[1]][axis_y]], color, alpha=0.8)
    return plt

def plot_fig_head(body_dict,plt,axis_x=0,axis_y=1,color='R'):
    cols=['l_eyes','l_ear','r_eyes','r_ear','nose',]
    axt = ['x', 'y', 'z']
    joint_ids=[[0,1],[2,3],[0,2],[0,4],[2,4],]
    joint_pos = []
    for col in cols:
        box=[]
        for ax in axt:
            box.append(body_dict[col][ax])
        joint_pos.append(box)
    
    # body plot
    for pos in joint_pos:
        #plt.scatter(pos[axis_x],pos[axis_y],marker='o',c=color,alpha=0.1)
        plt.scatter(pos[axis_x],pos[axis_y],marker='o',c=color,alpha=0.2)
    for box in joint_ids:       
        #plt.plot([joint_pos[box[0]][axis_x], joint_pos[box[1]][axis_x]],
        #         [joint_pos[box[0]][axis_y], joint_pos[box[1]][axis_y]], color, alpha=0.05)
        plt.plot([joint_pos[box[0]][axis_x], joint_pos[box[1]][axis_x]],
                 [joint_pos[box[0]][axis_y], joint_pos[box[1]][axis_y]], color, alpha=0.5)
    return plt

def plot_fig_look_zy(body_status_dict,look_points_dict,plt,axis_x=2,axis_y=1,color='R'):
    bace_x = body_status_dict['face_direction_bace_x'] 
    bace_y = body_status_dict['face_direction_bace_y']
    bace_z = body_status_dict['face_direction_bace_z']

    bace = [bace_x, bace_y, bace_z]
    
    look_center =[look_points_dict['look_point_center_x'],
                  look_points_dict['look_point_center_y'],
                  look_points_dict['look_point_center_z'],
                  look_points_dict['look_point_center_object'],
                  look_points_dict['look_point_center_distance'],
                 'look_center',]

    # look point plot
    # center
    plt.scatter(look_center[axis_x],look_center[axis_y],marker='.',c=color,alpha=0.5)
    
    #plt.plot([look_center[axis_x], bace[axis_x]],[look_center[axis_y], bace[axis_y]],
    #         color, alpha=0.01)
    plt.plot([look_center[axis_x], bace[axis_x]],[look_center[axis_y], bace[axis_y]],
             color, alpha=0.5)

    return plt,look_center

def plot_confidence_fix(df_data, title, joint_list,head):
    # confidenc
    #title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_head_confidence'
    cols=[]
    offsets=[]
    colors=[]
    
    #joint_list=[29,28,27,30,31]
    joint_name_list = [
            ['pelvis', 0],
            ['naval', 1],
            ['chest', 2],
            ['neck', 3],
            ['l_clavicle', 4],
            ['l_shoulder', 5],
            ['l_elbow', 6],
            ['l_wrist', 7],
            ['l_hand', 8],
            ['l_handtip', 9],
            ['l_thumb', 10],
            ['r_clavicle', 11],
            ['r_shoulder', 12],
            ['r_elbow', 13],
            ['r_wrist', 14],
            ['r_hand', 15],
            ['r_handtip', 16],
            ['r_thumb', 17],
            ['l_hip', 18],
            ['l_knee', 19],
            ['l_ankle', 20],
            ['l_foot', 21],
            ['r_hip', 22],
            ['r_knee', 23],
            ['r_ankle', 24],
            ['r_foot', 25],
            ['head', 26],
            ['nose', 27],
            ['l_eyes', 28],
            ['l_ear', 29],
            ['r_eyes', 30],
            ['r_ear', 31],]
    cm_name = 'jet'# B->G->R
    cm = plt.get_cmap(cm_name,len(joint_list))
    for ic,ii in enumerate(joint_list):
        #cols.append('data_skeletons_0_joints_{}_confidence_0'.format(ii))
        cols.append(head+joint_name_list[ii][0]+'_confidence')
        offsets.append(ic*3)
        colors.append(cm(ic))
    
    df = df_data.copy()
    plt.figure(figsize=(17,5))
    for col, color,offset,joint_no in zip(cols,colors,offsets,joint_list):
        plt.plot(df[col]+offset,label=joint_name_list[joint_no][0],color=color)
        
    plt.title(title)
    plt.legend()
    plt.grid()
    #plt.ylabel('x ')
    #plt.ylim(600,-1700)
    return plt

def get_pallet():
    cmap = plt.get_cmap("tab10")
    cmap = [cmap(0)[0:3],cmap(1)[0:3],cmap(2)[0:3],cmap(3)[0:3],cmap(4)[0:3],cmap(5)[0:3],
            cmap(6)[0:3],cmap(7)[0:3],cmap(8)[0:3],cmap(9)[0:3],cmap(10)[0:3],
            cmap(11)[0:3],cmap(12)[0:3],cmap(13)[0:3],cmap(14)[0:3],cmap(15)[0:3],
           ]
    cbox=[]
    for c in cmap:
        color='#'+format(int(c[0]*255), '02x')+format(int(c[1]*255), '02x')+format(int(c[2]*255), '02x')
        cbox.append(color)
    cmap = cbox
    return cmap

def get_true_pos(tbox,ids):
    target_lt_x = tbox[3] 
    target_lt_y = tbox[4]
    target_lt_z = tbox[5]
    target_lb_x = tbox[6] 
    target_lb_y = tbox[7]
    target_lb_z = tbox[8]
    target_rt_x = tbox[9] 
    target_rt_y = tbox[10]
    target_rt_z = tbox[11]
    target_rb_x = tbox[12] 
    target_rb_y = tbox[13]
    target_rb_z = tbox[14]
    target_id=ids
    if(target_id==5):
        true_x=(target_lt_x+target_rb_x)/2
        true_y=(target_lt_y+target_rb_y)/2
    elif(target_id==1):
        true_x=target_rt_x
        true_y=target_lb_y
    elif(target_id==2):
        true_x=(target_lt_x+target_rb_x)/2
        true_y=target_rb_y
    elif(target_id==3):
        true_x=target_lb_x
        true_y=target_lb_y
    elif(target_id==4):
        true_x=target_rt_x
        true_y=(target_lt_y+target_rb_y)/2
    elif(target_id==6):
        true_x=target_lt_x
        true_y=(target_lt_y+target_rb_y)/2
    elif(target_id==7):
        true_x=target_rb_x
        true_y=target_lt_y
    elif(target_id==8):
        true_x=(target_lb_x+target_rb_x)/2
        true_y=target_rt_y
    elif(target_id==9):
        true_x=target_lb_x
        true_y=target_rt_y
    else:
        true_x=(target_lt_x+target_rb_x)/2
        true_y=(target_lt_y+target_rb_y)/2 
    true_z=(target_lt_z+target_rb_z)/2
    return true_x,true_y,true_z


In [ ]:
def __convert_corner_list_to_homography_param(point_list):
    # type: (np.ndarray) -> np.ndarray
    """
    点の集合（dim * n の配列）を同次座標系に変換する
    :param point_list: ホモグラフィ行列変換用に整形を行う座標リスト
    :return: ホモグラフィ変換用行列
    """
    return np.vstack((point_list, np.ones((1, point_list.shape[1]))))

def __normalize(point_list):
    # type: (np.ndarray) -> (np.ndarray, np.ndarray)
    """
    正規化処理
    :param point_list: 正規化対象の座標リスト
    :return: 正規化結果, 正規化に用いた係数行列(理解が怪しい)
    """
    m = np.mean(point_list[:2], axis=1)
    max_std = max(np.std(point_list[:2], axis=1)) + 1e-9
    c = np.diag([1 / max_std, 1 / max_std, 1])
    c[0][2] = -m[0] / max_std
    c[1][2] = -m[1] / max_std
    return np.dot(c, point_list), c

def calculate_homography_matrix(origin, dest):
    # type: (np.ndarray, np.ndarray) -> np.ndarray
    """
    線形DLT法にて、 変換元を変換先に対応づけるホモグラフィ行列を求める。先行実装に倣う。
    :param origin: ホモグラフィ行列計算用の初期座標配列
    :param dest: ホモグラフィ行列計算用の移動先座標配列
    :return: 計算結果のホモグラフィ行列(3 x 3)
    """
    assert origin.shape == dest.shape

    origin = __convert_corner_list_to_homography_param(origin.T)
    dest = __convert_corner_list_to_homography_param(dest.T)

    # 点を調整する（数値計算上重要）
    origin, c1 = __normalize(origin)  # 変換元
    dest, c2 = __normalize(dest)      # 変換先
    # 線形法計算のための行列を作る。
    nbr_correspondences = origin.shape[1]
    a = np.zeros((2 * nbr_correspondences, 9))
    for i in range(nbr_correspondences):
        a[2 * i] = [-origin[0][i], -origin[1][i], -1, 0, 0, 0, dest[0][i] * origin[0][i], dest[0][i] * origin[1][i],
                    dest[0][i]]
        a[2 * i + 1] = [0, 0, 0, -origin[0][i], -origin[1][i], -1, dest[1][i] * origin[0][i], dest[1][i] * origin[1][i],
                        dest[1][i]]
    u, s, v = np.linalg.svd(a)
    homography_matrix = v[8].reshape((3, 3))
    homography_matrix = np.dot(np.linalg.inv(c2), np.dot(homography_matrix, c1))
    homography_matrix = homography_matrix / homography_matrix[2, 2]
    return homography_matrix

# 射影変換
def get_trans_met(origin_corner_list,destination_corner_list):
    """
        :param img: 元画像
        :param origin_corner_list: 切り出し対象物の4角の座標リスト [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
        :param destination_corner_list: 対象物の移動先となるの4角の座標リスト
        :return: 変換後の画像
    """
    # origin_corner_list=[[1,1],[1,0],[0,0],[0,1]]
    # destination_corner_list=[[2,2],[2,0],[0,0],[0,2]]
    # ホモグラフィ変換に必要な切り出し座標・変換先座標のパラメータを生成。
    origin_corner_list = np.array(origin_corner_list)
    destination_corner_list = np.array(destination_corner_list)

    # ホモグラフィ変換行列(変換先 -> 変換元 の変換用)を取得
    # いずれの切り出し座標にも通じる、変換後の座標との関係を示す数式(行列)を求めるイメージ。
    homography_matrix = calculate_homography_matrix(origin_corner_list, destination_corner_list)
    inv_homography_matrix = np.linalg.inv(homography_matrix)
    inv_homography_matrix /= inv_homography_matrix[2, 2]
    
    return homography_matrix

def trans_point(homography_matrix, point):
    points=list(point)+[1]
    convert_points=np.dot(homography_matrix,points)
    convert_point=convert_points[0:2]
    return convert_point

In [ ]:
# 射影変換 sample
origin_corner_list=[[1,1],[1,0],[0,0],[0,1]]
destination_corner_list=[[2,2],[2,-2],[-2,-2],[-2,2]]
homography_matrix = get_trans_met(origin_corner_list,destination_corner_list)

point=[1,0]
convert_point = trans_point(homography_matrix, point)
print(convert_point)

### データ読み込み  

In [ ]:
joint_name_list = [
        ['pelvis', 0],
        ['naval', 1],
        ['chest', 2],
        ['neck', 3],
        ['l_clavicle', 4],
        ['l_shoulder', 5],
        ['l_elbow', 6],
        ['l_wrist', 7],
        ['l_hand', 8],
        ['l_handtip', 9],
        ['l_thumb', 10],
        ['r_clavicle', 11],
        ['r_shoulder', 12],
        ['r_elbow', 13],
        ['r_wrist', 14],
        ['r_hand', 15],
        ['r_handtip', 16],
        ['r_thumb', 17],
        ['l_hip', 18],
        ['l_knee', 19],
        ['l_ankle', 20],
        ['l_foot', 21],
        ['r_hip', 22],
        ['r_knee', 23],
        ['r_ankle', 24],
        ['r_foot', 25],
        ['head', 26],
        ['nose', 27],
        ['l_eyes', 28],
        ['l_ear', 29],
        ['r_eyes', 30],
        ['r_ear', 31],]

In [ ]:
# 領域分割しきい値
def get_sep_box(pp='p21',ss='s03',is_mask=0):
    if(pp=='p04' and ss=='s03' and is_mask==0):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 535, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 840, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1300,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2230,2245, 3],[2275,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2520,2545, 4],[2580,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2900,2930, 3],[2960,2990, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3260,3285, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3700, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        #continue
    elif(pp=='p04' and ss=='s03' and is_mask==1):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 530, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 840, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1290,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2510,2545, 4],[2590,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2910,2940, 3],[2965,2995, 2],[3030,3060, 1],[3080,3120, 4],
            [3140,3180, 7],[3200,3240, 8],[3270,3295, 9],[3340,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3690, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        #continue
    elif(pp=='p04' and ss=='s02' and is_mask==0):#
        front_box=[ 
            [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 530, 600, 3],[ 630, 700, 9],[ 740, 810, 7],[ 850, 900, 1],[ 930,1000, 5],
            [1190,1250, 5],[1290,1350, 3],[1390,1460, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2000,2070, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2500, 7],[2510,2545, 4],[2590,2610, 1],[2630,2670, 5],
            
            [2810,2880, 5],[2900,2920, 3],[2960,3000, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3260,3285, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3655,3700, 5],[3865,3880, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        #continue
    elif(pp=='p04' and ss=='s02' and is_mask==1):#
        front_box=[ 
             [  0,  310, 0],
            [ 330, 400, 5],[ 430, 500, 1],[ 540, 600, 3],[ 630, 700, 9],[ 765, 805, 7],[ 860, 900, 1],[ 950,1000, 5],
            [1200,1250, 5],[1290,1350, 3],[1390,1450, 1],[1490,1550, 7],[1590,1660, 9],[1680,1750, 3],[1790,1860, 5],
            [2035,2050, 5],[2095,2125, 1],[2150,2180, 2],[2220,2250, 3],[2270,2310, 6],[2330,2350, 9],
            [2380,2430, 8],[2450,2480, 7],[2510,2550, 4],[2580,2610, 1],[2630,2670, 5],
            
            [2810,2860, 5],[2910,2940, 3],[2970,3000, 2],[3020,3050, 1],[3080,3110, 4],
            [3150,3180, 7],[3200,3240, 8],[3265,3280, 9],[3320,3360, 6],[3385,3420, 3],[3450,3480, 5],
            
            [3660,3690, 5],[3865,3900, 5],[4080,4110, 5],[4280,4300, 5],
            [4310,4460, 0],[4460,4500, 0],
                  ]
        # fit 2
        #continue
    else:
        # 時刻
        front_box=[
            [  0,  300, 0],
                   [ 300, 400, 5],
                   [ 400, 500, 1],
                   [ 500, 600, 3],
                   [ 600, 700, 9],
                   [ 700, 800, 7],
                   [ 800, 900, 1],
                   [ 900,1000, 5],
                   
                   [1150,1250, 5],
                   [1250,1350, 3],
                   [1350,1450, 1],
                   [1450,1550, 7],
                   [1550,1650, 9],
                   [1650,1750, 3],
                   [1750,1850, 5],
                   
                   [2000,2060, 5],
                   [2060,2120, 1],
                   [2120,2180, 2],
                   [2180,2240, 3],
                   [2240,2300, 6],
                   [2300,2360, 9],
                   [2360,2420, 8],
                   [2420,2480, 7],
                   [2480,2540, 4],
                   [2540,2600, 1],
                   [2600,2660, 5],
                   
                   [2810,2870, 5],
                   [2870,2930, 1],
                   [2930,2990, 2],
                   [2990,3050, 3],
                   [3050,3110, 6],
                   [3110,3170, 9],
                   [3170,3230, 8],
                   [3230,3290, 7],
                   [3290,3350, 4],
                   [3350,3410, 1],
                   [3410,3470, 5],
                   
                   [3620,3680, 5],
                   [3830,3890, 5],
                   [4040,4100, 5],
                   [4250,4310, 5],
                   [4310,4460, 0],
                   [4460,4500, 0],
                  ]
        #continue
    return front_box

In [ ]:
# アノテーション済みデータ読み込み
exp_code = "01_20201012_exp_fix3"
finename = save_pkl_dir+'/'+exp_code+'.pkl'
with open(finename, 'rb') as web:
      data_box = pickle.load(web)
      #print techacademy
print(data_box[0][0:5],data_box[0][6:7],len(data_box[0]))

In [ ]:
# 再計算
# 改良アルゴの再計算
exp_code = "02_20201012_exp"
data_box2=[]
for ii in range(len(data_box)):
    box=data_box[ii]
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=box[6]
    if(pp=='p20'):
        continue
    
#     if not (ss=='s02'):
#         continue
    if(ss=='s02'):
        look_points=look_points2
    else:
        look_points=look_points3
    print(pp,ss,is_mask)

    df = df_data.copy()

    if('s03'==ss):
        offset_x=-20
        offset_y=-150
        offset_z=0
    elif('s02'==ss):
        offset_x=-50
        offset_y=-110
        offset_z=60

    head='data_body_raw_'
    joint_name_list = [
        ['pelvis', 0],
        ['naval', 1],
        ['chest', 2],
        ['neck', 3],
        ['l_clavicle', 4],
        ['l_shoulder', 5],
        ['l_elbow', 6],
        ['l_wrist', 7],
        ['l_hand', 8],
        ['l_handtip', 9],
        ['l_thumb', 10],
        ['r_clavicle', 11],
        ['r_shoulder', 12],
        ['r_elbow', 13],
        ['r_wrist', 14],
        ['r_hand', 15],
        ['r_handtip', 16],
        ['r_thumb', 17],
        ['l_hip', 18],
        ['l_knee', 19],
        ['l_ankle', 20],
        ['l_foot', 21],
        ['r_hip', 22],
        ['r_knee', 23],
        ['r_ankle', 24],
        ['r_foot', 25],
        ['head', 26],
        ['nose', 27],
        ['l_eyes', 28],
        ['l_ear', 29],
        ['r_eyes', 30],
        ['r_ear', 31],]
    for col,no in joint_name_list:
        for i in range(len(df)):
            df.at[i,head+col+'_x']=df.at[i,head+col+'_x']+offset_x
            df.at[i,head+col+'_y']=df.at[i,head+col+'_y']+offset_y
            df.at[i,head+col+'_z']=df.at[i,head+col+'_z']+offset_z

    # imp
    bace_h = np.array([0.0, 0.0, 1.0])
    bace_v = np.array([0.0, 1.0, 0.0])
    # 補正
    look_points.set_offset(offset_h=0.0, offset_v=0.0)
    # 処理
    for i in range(len(df)):
        # 10 Hz 器官点データ取得
        body_dict = get_data_75_3_body(df, i, head='data_body_raw_')
        axt = ['x', 'y', 'z']
        l_eyes = np.zeros(3)
        r_eyes = np.zeros(3)
        nose = np.zeros(3)
        head = np.zeros(3)
        neck = np.zeros(3)
        r_ear = np.zeros(3)
        l_ear = np.zeros(3)
        for ax in range(3):
            l_eyes[ax] = body_dict['l_eyes'][axt[ax]]
            r_eyes[ax] = body_dict['r_eyes'][axt[ax]]
            nose[ax] = body_dict['nose'][axt[ax]]
            head[ax] = body_dict['head'][axt[ax]]
            neck[ax] = body_dict['neck'][axt[ax]]
            l_ear[ax] = body_dict['l_ear'][axt[ax]]
            r_ear[ax] = body_dict['r_ear'][axt[ax]]
        if(body_dict['l_eyes']['x']==0):
            continue
        # org
        is_front_ward = False
        h_toward_vector=(r_ear + r_eyes)/2. - (l_ear + l_eyes)/2.
        h_bace_vector=bace_h
        v_toward_vector=nose - (l_ear + r_ear)/2.
        v_bace_vector=bace_v
        # 再計算(顔向き)
        face_direction.Calculate_for_test(body_dict,
                                           h_toward_vector=h_toward_vector,
                                           h_bace_vector=h_bace_vector,
                                           v_toward_vector=v_toward_vector,
                                           v_bace_vector=v_bace_vector,
                                           is_data=True,
                                           is_front_ward=is_front_ward,)
        body_status_dict_r = face_direction.output_data
        df = set_direction_data(df,i,body_status_dict_r,'r01_')
        # 再計算(視点)
        look_points.Calculate(body_status_dict_r, is_data=True)
        look_points_dict_r= look_points.output_data
        df = set_look_data(df,i,look_points_dict_r,'r01_')
        
        # 02  
        is_front_ward = True
        h_toward_vector=nose - (l_ear + r_ear)/2.
        v_toward_vector=nose - (l_ear + r_ear)/2.
        v_bace_vector=bace_v
        #print(h_toward_vector,h_bace_vector,v_toward_vector,v_bace_vector)
        # 再計算(顔向き)
        face_direction.Calculate_for_test(body_dict,
                                           h_toward_vector=h_toward_vector,
                                           h_bace_vector=h_bace_vector,
                                           v_toward_vector=v_toward_vector,
                                           v_bace_vector=v_bace_vector,
                                           is_data=True,
                                           is_front_ward=is_front_ward,)
        body_status_dict_r = face_direction.output_data
        df = set_direction_data(df,i,body_status_dict_r,'r02_')
        # 再計算(視点)
        look_points.Calculate(body_status_dict_r, is_data=True)
        look_points_dict_r= look_points.output_data
        df = set_look_data(df,i,look_points_dict_r,'r02_') 
                
#         # 鉛直改善
#         # 12  
#         is_front_ward = True
#         h_toward_vector=nose - (l_ear + r_ear)/2.
#         v_toward_vector=nose - head
#         v_bace_vector=bace_v
#         #print(h_toward_vector,h_bace_vector,v_toward_vector,v_bace_vector)
#         # 再計算(顔向き)
#         face_direction.Calculate_for_test(body_dict,
#                                            h_toward_vector=h_toward_vector,
#                                            h_bace_vector=h_bace_vector,
#                                            v_toward_vector=v_toward_vector,
#                                            v_bace_vector=v_bace_vector,
#                                            is_data=True,
#                                            is_front_ward=is_front_ward,)
#         body_status_dict_r = face_direction.output_data
#         df = set_direction_data(df,i,body_status_dict_r,'r12_')
#         # 再計算(視点)
#         look_points.Calculate(body_status_dict_r, is_data=True)
#         look_points_dict_r= look_points.output_data
#         df = set_look_data(df,i,look_points_dict_r,'r12_')         
 
#         # 22  
#         is_front_ward = True
#         h_toward_vector=nose - (l_ear + r_ear)/2.
#         v_toward_vector=nose - neck
#         v_bace_vector=bace_v
#         #print(h_toward_vector,h_bace_vector,v_toward_vector,v_bace_vector)
#         # 再計算(顔向き)
#         face_direction.Calculate_for_test(body_dict,
#                                            h_toward_vector=h_toward_vector,
#                                            h_bace_vector=h_bace_vector,
#                                            v_toward_vector=v_toward_vector,
#                                            v_bace_vector=v_bace_vector,
#                                            is_data=True,
#                                            is_front_ward=is_front_ward,)
#         body_status_dict_r = face_direction.output_data
#         df = set_direction_data(df,i,body_status_dict_r,'r22_')
#         # 再計算(視点)
#         look_points.Calculate(body_status_dict_r, is_data=True)
#         look_points_dict_r= look_points.output_data
#         df = set_look_data(df,i,look_points_dict_r,'r22_')         
        
       
    # 結果格納
    sbox=[pp, ss, tt, st, fn, df, is_mask]
    data_box2.append(sbox)
    #  csv保存
    save_dir2=save_csv_dir +'/data_df_rx_r02'
    if not os.path.isdir(save_dir2):
        os.makedirs(save_dir2)
    # 
    filecode=save_dir2+'/recalc_'+pp+ss+'_'+str(is_mask)+"_r02.csv"
    df.to_csv(filecode, index=False)
    print(filecode)
data_box=data_box2

print('data_count: ',len(data_box))
finename = save_pkl_dir+'/'+exp_code+'.pkl'
with open(finename, 'wb') as web:
    pickle.dump(data_box, web)
print('saved ', finename)

finename = save_pkl_dir+'/'+exp_code+'.pkl'
with open(finename, 'rb') as web:
      data_box = pickle.load(web)
      #print techacademy


In [ ]:
# アノテーション済みデータ読み込み
exp_code = "02_20201012_exp"
finename = save_pkl_dir+'/'+exp_code+'.pkl'
with open(finename, 'rb') as web:
      data_box = pickle.load(web)
      #print techacademy
print(data_box[0][0:5],data_box[0][6:7],len(data_box[0]))

In [ ]:
# 位置補正　無補正
# 領域設定
xmax=max([514,-698])
xmin=min([514,-698])
ymax=max([163,847])
ymin=min([163,847])
zmax=max([733,757])
zmin=min([733,757])
side_target_box=side_block_9(xmax,xmin,ymax,ymin,zmax,zmin)

# 代表点算出
exp_code = "02_20201015_exp_r02_nofit"
save_dir = save_img_dir+'/'+exp_code
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

for alg in ['r02',
            ]:
    print(alg)
    for box in data_box:
        pp=box[0]
        ss=box[1]
        tt=box[2]
        st=box[3]
        fn=box[4]
        df_data=box[5]
        is_mask=int(box[6])
        # データ確認
        df = df_data.copy()
        # 10色パレット
        cmap = get_pallet()

        # 時刻
        #front_box=get_sep_box(pp=pp,ss=ss,is_mask=is_mask) 

        # 絶対
        plt.figure(figsize=(14,8))
        ax = plt.axes()
        
        #
        box5=[]
        calced_data_box=[]
        calced_st_box=[]
        calced_col_box=[]
        # side視点分布
        title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_side_look_mask_'+str(is_mask)+'_'+alg
        colx=alg+'_look_point_center_x'
        coly=alg+'_look_point_center_y'
        # 修正演算
        x1=[0,0]
        x2=[0,0]
        x3=[0,0]
        x4=[0,0]
        cx1=[0,0]
        cx2=[0,0]
        cx3=[0,0]
        cx4=[0,0]
        
        for ids in range(1,10):
            df = df_data.copy()  
            c=cmap[ids]
            df=df[df['stable_gaze_flag']==ids]            
            df=df.reset_index(drop=True)
            if(ids==0):
                continue
            tbox=side_target_box[ids-1]
            true_x,true_y,true_z = get_true_pos(tbox,ids)
            df['true_x'] = float(true_x) 
            df['true_y'] = float(true_y)
            df['true_z'] = float(true_z)
            if(ids==3):
                x1=[np.nanmean(df[colx]),np.nanmean(df[coly])]
                cx1=[true_x,true_y]
            elif(ids==9):
                x2=[np.nanmean(df[colx]),np.nanmean(df[coly])]
                cx2=[true_x,true_y]
            elif(ids==7):
                x3=[np.nanmean(df[colx]),np.nanmean(df[coly])]
                cx3=[true_x,true_y]
            elif(ids==1):
                x4=[np.nanmean(df[colx]),np.nanmean(df[coly])]
                cx4=[true_x,true_y]
        origin_corner_list=[x4,x3,x2,x1]
        destination_corner_list=[cx4,cx3,cx2,cx1]
        homography_matrix = get_trans_met(origin_corner_list,destination_corner_list)
        #print(origin_corner_list)
        #print(destination_corner_list)
        
        # 本計算
        mean5_x=0
        mean5_y=0
        is_5=False
        # look vecter
        for ids in range(1,10):
            df = df_data.copy()  
            c=cmap[ids]
            df=df[df['stable_gaze_flag']==ids] 
            #print(ids,len(df))
            df=df.reset_index(drop=True)
            if(ids==0):
                continue
            tbox=side_target_box[ids-1]
            target_lt_x = tbox[3] 
            target_lt_y = tbox[4]
            target_lt_z = tbox[5]
            target_lb_x = tbox[6] 
            target_lb_y = tbox[7]
            target_lb_z = tbox[8]
            target_rt_x = tbox[9] 
            target_rt_y = tbox[10]
            target_rt_z = tbox[11]
            target_rb_x = tbox[12] 
            target_rb_y = tbox[13]
            target_rb_z = tbox[14]
            true_x,true_y,true_z = get_true_pos(tbox,ids)
            df['true_x'] = float(true_x) 
            df['true_y'] = float(true_y)
            df['true_z'] = float(true_z)

#             # 視点座標補正
#             for j in range(len(df)):
#                 inx = df.at[j,colx]
#                 iny = df.at[j,coly]
#                 point=[inx,iny]
#                 convert_point = trans_point(homography_matrix, point)
#                 df.at[j,colx]=convert_point[0]
#                 df.at[j,coly]=convert_point[1]            
            #if(b[2]==5):
#             if(b[2]==5 and is_5==False):
#                 mean5_x=np.nanmean(df[colx])-true_x
#                 mean5_y=np.nanmean(df[coly])-true_y
            df[colx]=df[colx]-mean5_x
            df[coly]=df[coly]-mean5_y
            #
            
            r=111
            if not (ids==5):                
                draw_circle = plt.Circle((true_x, true_y), r ,fill=False, color=c, alpha=0.8)
                plt.gcf().gca().add_artist(draw_circle)
                plt.plot([target_lt_x, target_lb_x],[target_lt_y, target_lb_y], c, alpha=0.2)
                plt.plot([target_lb_x, target_rb_x],[target_lb_y, target_rb_y], c, alpha=0.2)
                plt.plot([target_rb_x, target_rt_x],[target_rb_y, target_rt_y], c, alpha=0.2)
                plt.plot([target_rt_x, target_lt_x],[target_rt_y, target_lt_y], c, alpha=0.2)
            elif not (is_5):
                is_5=True
                draw_circle = plt.Circle((true_x, true_y), r ,fill=False, color=c, alpha=0.8)
                plt.gcf().gca().add_artist(draw_circle)
                plt.plot([target_lt_x, target_lb_x],[target_lt_y, target_lb_y], c, alpha=0.2)
                plt.plot([target_lb_x, target_rb_x],[target_lb_y, target_rb_y], c, alpha=0.2)
                plt.plot([target_rb_x, target_rt_x],[target_rb_y, target_rt_y], c, alpha=0.2)
                plt.plot([target_rt_x, target_lt_x],[target_rt_y, target_lt_y], c, alpha=0.2)
                
            x=df[colx]
            y=df[coly]
            plt.scatter(x,y,label=str(ids),color=c,alpha=0.4)
            
        plt.title(title)
        # plt.xlim(800,-1400)
        # plt.ylim(-200,1200)
        plt.xlim(1000,-1800)
        plt.ylim(-100,1500)
        plt.xlabel(' x [mm]')
        plt.ylabel(' y [mm]')
        handles, labels = plt.gca().get_legend_handles_labels()
        by_label = OrderedDict(zip(labels, handles))
        plt.legend(by_label.values(), by_label.keys(),loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_dir, save_file)
        plt.savefig(save_path)
        plt.show()    